encoder transformer层的linear2层（etl2）替换为近似矩阵乘法

In [1]:
import numpy as np
import os
import sys
# 获取当前文件所在的文件夹路径
if "__file__" in globals():
    # 获取__file__变量的值
    file_path = __file__
    # 获取当前文件所在的文件夹路径
    dir_now = os.path.dirname(file_path)
else:
    # 获取当前工作目录
    dir_now = os.getcwd()
sys.path.append(dir_now)
sys.path.append(os.path.join(dir_now, '../'))
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # 防止jupyter爆内存
import matmul as mm
import math_util as mu
from NNutils import *
# import scipy.io as io
from amm_methods import *

/shared/conda/envs/bolt/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_PLUTO
# method = METHOD_MITHRALPQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
# for method in [METHOD_MITHRAL, METHOD_PQ]:

In [3]:
linear_name = 'etl2'
feedback_bits = 256
linear_name_full = "ex_linear2"

auto_train_change_nbits = False # 是否根据已运行的训练性能结果改变nbits自动训练，（train_sam_num取已训练的最大值）
auto_train_change_upcast = False # 是否根据已运行的训练性能结果改变upcast自动训练，（train_sam_num取已训练的最大值）

if auto_train_change_nbits:
    nbits_trained = 8
if auto_train_change_upcast:
    upcast_trained = 16
quantize_lut = True
nbits_goal = 16
upcast_goal = 16
lut_work_const = -1
del0_trained = True
del0_goal = True
if quantize_lut == False:
    nbits_goal = 0
nbits = nbits_goal # 要运行的量化比特数
upcast_every = upcast_goal # 要运行的upcast
del0 = del0_goal # 要运行的del0

test_sam_num = 1000 # 测试集样本数(如需修改，请同时修改下面的读取文件，现文件默认1000个样本)

if not auto_train_change_nbits and not auto_train_change_upcast:
    ncodebooks = 256 # max:512
    ncentroids = 128
    train_sam_num = 75 # 训练集样本数
elif auto_train_change_nbits:
    param2change = "nbits"
    param_trained = nbits_trained
    param_goal = nbits_goal
    cb_ct_ntr_combinations_unique = change_param_auto_run_list(linear_name, method, feedback_bits, param2change, param_trained, param_goal, "upcast_every", 16)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])
elif auto_train_change_upcast:
    param2change = "upcast_every"
    param_trained = upcast_trained
    param_goal = upcast_goal
    cb_ct_ntr_combinations_unique = change_param_auto_run_list(linear_name, method, feedback_bits, param2change, param_trained, param_goal, "nbits", 8)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])

batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [4]:
AMM_train_dirs = get_AMM_train_dirs(linear_name, linear_name_full, method, feedback_bits, train_sam_num, test_sam_num)
create_dir(AMM_train_dirs["dir_result"])

In [5]:
dataset_prepare(AMM_train_dirs["dir_joined"], linear_name_full, feedback_bits, [train_sam_num, test_sam_num], 
                batch_size, S1 = S1_dict[linear_name])

In [6]:
if method == METHOD_PLUTO:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut, 
                        upcast_every=upcast_every, bias_path=AMM_train_dirs["biaspath"],lut_work_const=-1)
elif method == METHOD_MITHRAL:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut,
                        upcast_every=upcast_every, lut_work_const=lut_work_const, del0=del0)
else:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut,
                        upcast_every=upcast_every)

running method:  Mithral
X.shape:  (76800, 512)
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    14.627148500077402
learn_multisplits(): returning loss:  0.07845373610408463
multisplits: [Multisplit(
	dim=0, scaleby=1024.0, offset=0.03772098571062088,
vals=array([38], dtype=int32), Multisplit(
	dim=0, scaleby=512.0, offset=1.2577511370182037e-06,
vals=array([ 0, 85], dtype=int32), Multisplit(
	dim=1, scaleby=1024.0, offset=0.0,
vals=array([111,  94,  12,   0], dtype=int32), Multisplit(
	dim=0, scaleby=512.0, offset=0.0,
vals=array([  0,   0,  18,   0,  59,  50, 120,   0], dtype=int32), Multisplit(
	dim=1, scaleby=512.0, offset=0.0,
vals=array([21,  0, 97,  0, 11, 12,  0,  0,  0,  0, 15, 29,  0,  0,  0,  0],
      dtype=int32), Multisplit(
	dim=0, scaleby=512.0, offset=0.0,
vals=array([  0,   0,   0,   0,   0,   0,   0,   0,   8,  12,  27,   0,   0,
         0,   0,   0,  48,   0,  71,   0,   0,  44,   0,   0, 101,   0,
       143,   0,   0,   0,   0,   0

/home/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../clusterize.py:869: RuntimeWarning: divide by zero encountered in double_scalars
  scale = 254. / upper_val


split err: upper_val0.0 offset0.0
  oldvals:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
  newvals:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
split err: upper_val0.0 offset0.0
  oldvals:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  newvals:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
learn_multisplits(): returning loss:  2.911051649716907e-06
multisplits: [Multisplit(
	dim=40, scaleby=1024.0, offset=0.04717966169118881,
vals=array([48], dtype=int32), Multisplit(
	dim=40, scaleby=1024.0, offset=0.017592504620552063,
vals=array([ 18, 193], dtype=int32), Multisplit(
	dim=40, scaleby=512.0, offset=0.006487185135483742,
vals=array([  3,  24,  74, 121], dtype=int32), Multisplit(
	di

In [7]:
x_test = np.load(AMM_train_dirs["dir_test"]+'/'+AMM_train_dirs["linearin_path_test"])
w_test = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["weightpath"])
bias = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["biaspath"])
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
if method == METHOD_PLUTO:
    y_out_last = y_out_matmul
else:
    y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [8]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), 
                                                        y_out_last_re.astype(np.float32))
elif method == METHOD_MITHRAL:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i_ql%i_nb%i_uc%i_lwc%i_del0%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids, quantize_lut, nbits, upcast_every, lut_work_const, del0)), y_out_last_re)
elif method == METHOD_PLUTO:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i_ql%i_nb%i_uc%i_lwc%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids, quantize_lut, nbits, upcast_every, lut_work_const)), y_out_last_re)
elif method == METHOD_PQ or method == METHOD_MITHRALPQ:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i_ql%i_nb%i_uc%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids, quantize_lut, nbits, upcast_every)), y_out_last_re)
else:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids)), y_out_last_re)

[[ 5.6432509e-01  2.6905350e-04  5.6659937e-01 ...  2.9738379e-01
   6.1344199e-02  5.7909352e-01]
 [ 4.9596521e-01  3.2495864e-02  5.5683368e-01 ...  2.5441471e-01
   1.1539131e-02  5.0389767e-01]
 [ 4.7545722e-01  3.1987634e-03  5.3339601e-01 ...  2.7296954e-01
   4.6695650e-02  5.2831191e-01]
 ...
 [ 4.6862125e-01  1.1745746e-01  2.5897986e-01 ...  2.8371179e-01
   2.1466568e-01  3.0760711e-01]
 [ 3.5143286e-01 -1.3645074e-01  1.2616634e-01 ...  3.4816542e-01
   1.0236014e-01  3.9549839e-01]
 [ 3.5729226e-01  5.3980403e-02  2.2870618e-01 ...  3.8592745e-02
   1.3068067e-01  2.4510662e-01]]
y_out_last.shape:  (1024000, 64)
y_out_last_re.shape:  (1000, 32, 32, 64)
